# Overall Pipeline

간단한 파이프라인은 아래와 같이 구성됩니다.

In [1]:
%load_ext lab_black
# !pip install lxml
# !pip install beautifulsoup4
# !pip install hgtk
# !pip install jamo
# !pip install Unidecode
# !pip install inflect
# !pip install librosa
# !pip install matplotlib
# !sudo apt-get install libsndfile1

In [2]:
import os
import sys
import json
import requests
import re
import urllib.request
import soundfile as sf

import sys

sys.path.insert(0, "./voice_synthesis/tacotron2")
sys.path.insert(0, "./voice_synthesis/waveglow")
from summarization.inference import get_summarized_text
from voice.preprocessing.kor_transliterator import Transliterator
from voice_synthesis.inference import Synthesizer

from bs4 import BeautifulSoup

using cached model
using cached model


In [10]:
class VoiceSummarizer:
    """ Simple Voice Summarizer Demo """

    def __init__(self, checkpoint_path):
        self.ckpt_kobart = checkpoint_path["kobart"]
        self.ckpt_tacotron = checkpoint_path["tacotron"]
        self.ckpt_waveglow = checkpoint_path["waveglow"]

        assert self.ckpt_kobart, "You should enter KoBART checkpoint path."
        assert self.ckpt_tacotron, "You should enter Tacotron checkpoint path."
        assert self.ckpt_waveglow, "You should enter Waveglow checkpoint path."

    def get_news_content(self, url):
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "lxml")
        content = ""

        if re.search(r"daum", url) != None:
            tag = "div#harmonyContainer p"
        elif re.search(r"yna", url) != None:
            tag = "div[class=scroller01] p"
        elif re.search(r"joins", url) != None:
            tag = "div#article_body"

        for p in soup.select(tag):
            content += p.get_text()
        return content

    def summarize(self, text: str) -> str:
        return get_summarized_text(self.ckpt_kobart, text)

    def transliterate(self, text: str) -> str:
        text = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘|\(\)\[\]\<\>`'…》]", " ", text)
        text = text.replace("  ", " ")
        return Transliterator(text).transliterate()

    def synthesize(self, text: str) -> None:
        synthesizer = Synthesizer(self.ckpt_tacotron, self.ckpt_waveglow)
        audio, sampling_rate = synthesizer.inference(text)
        sf.write("./summary.wav", audio, sampling_rate)

    def run(self, url):
        news = self.get_news_content(url)
        summarized = self.summarize(news)
        transliterated = self.transliterate(summarized)
        self.synthesize(transliterated)

        return news, summarized, transliterated

In [4]:
search_url = "https://www.yna.co.kr/view/AKR20210610125900057?section=industry/all"
checkpoint_path = {
    "kobart": "./summarization/checkpoints/version-16/best_model_step_40730_loss_1.8838.pt",
    "tacotron": "./voice_synthesis/tacotron2/output/checkpoint_680000",
    "waveglow": "./voice_synthesis/waveglow/checkpoints/waveglow_1514000",
}

In [5]:
summarizer = VoiceSummarizer(checkpoint_path)

In [6]:
original, summary, transliterated = summarizer.run(search_url)

Warning! Reached max decoder steps


In [7]:
original

"[울산대 제공. 재판매 및 DB 금지] (울산=연합뉴스) 김용태 기자 = 울산대학교는 10일 교육부의 '대학 중점 연구소 지원 사업'에 2개 연구소가 선정됐다고 밝혔다. 울산대에 따르면 선정 연구소는 화학산업종합연구소(연구책임 화학과 이영일 교수)와 소화기병연구소(연구책임 의학과 명승재 교수)다. 이들 연구소는 올해부터 2030년 5월까지 9년 동안 해마다 사업비 7억7천만원을 교육부로부터 각각 지원받아 연구를 진행한다. 화학산업종합연구소는 '산학 융합 기반 신산업 창출형 나노 화학 소재 연구개발'을 목표로 하고 있다. 세부 연구 주제는 신규 촉매 소재 개발, 정밀 화학 소재 개발, 스마트 에너지 소재 개발 등이다. 지역 화학업체와 유기적으로 교류해 산학 융합형 사업을 운영하고, 화학산업 연구개발 분야 전문 인력을 양성한다. 소화기병연구소는 소화기 질환 신약과 진단법을 개발하는 '다중오믹스 기반 소화기 질환 진단 및 치료 기술' 개발을 목표로 한다. 세부 과제는 임상 기반 신약 개발 연구, 다중오믹스 기반 연구, 동물 모델 기반 기전 연구 등이다. 소화기 질환 연구 인프라를 구축해 국내외 경쟁력을 확보하고, 해당 질환의 중개 연구와 신약 개발에 특화된 인력을 양성한다. yongtae@yna.co.kr\n<저작권자(c) 연합뉴스,\n무단 전재-재배포 금지>\n2021/06/10 15:35 송고\n"

In [8]:
summary

"울산대학교는 10일 교육부의 '대학 중점 연구소 지원 사업'에 화학산업종합연구소와 소화기병연구소가 선정되어 올해부터 2030년 5월까지 9년 동안 해마다 사업비 7억7천만원을 교육부로부터 각각 지원받아 연구를 진행한다."

In [9]:
transliterated

'울산대학교는 십일 교육부의 대학 중점 연구소 지원 사업 에 화학산업종합연구소와 소화기병연구소가 선정되어 올해부터 이천삼십년 오월까지 구년 동안 해마다 사업비 칠억칠천만원을 교육부로부터 각각 지원받아 연구를 진행한다 '